In [5]:
import sympy as sp

# Define symbolic variables for Euler angles and tensor coefficients
alpha, beta, gamma = sp.symbols('alpha beta gamma', real=True)
iso, delta_xx, delta_yy, delta = sp.symbols('iso delta_xx delta_yy delta', real=True)

# Define the original coefficients T_{2m}
T_2m = {
    -2: 0.5 * (delta_xx - delta_yy),
    -1: 0,
    0: sp.sqrt(3/2) * delta,
    1: 0,
    2: 0.5 * (delta_xx - delta_yy)
}

# Reduced Wigner d-matrix elements for l=2
d_2 = sp.Matrix([
    [0.25 * (1 + sp.cos(beta))**2, 0.5 * sp.sin(beta) * (1 + sp.cos(beta)), sp.sqrt(6)/4 * sp.sin(beta)**2, 
     0.5 * sp.sin(beta) * (1 - sp.cos(beta)), 0.25 * (1 - sp.cos(beta))**2],
    [-0.5 * sp.sin(beta) * (1 + sp.cos(beta)), sp.cos(beta) * (1 + sp.cos(beta)), sp.sqrt(6)/2 * sp.sin(beta) * sp.cos(beta), 
     sp.cos(beta) * (1 - sp.cos(beta)), -0.5 * sp.sin(beta) * (1 - sp.cos(beta))],
    [sp.sqrt(6)/4 * sp.sin(beta)**2, -sp.sqrt(6)/2 * sp.sin(beta) * sp.cos(beta), 0.5 * (3 * sp.cos(beta)**2 - 1), 
     sp.sqrt(6)/2 * sp.sin(beta) * sp.cos(beta), sp.sqrt(6)/4 * sp.sin(beta)**2],
    [-0.5 * sp.sin(beta) * (1 + sp.cos(beta)), -sp.cos(beta) * (1 + sp.cos(beta)), sp.sqrt(6)/2 * sp.sin(beta) * sp.cos(beta), 
     -sp.cos(beta) * (1 - sp.cos(beta)), 0.5 * sp.sin(beta) * (1 - sp.cos(beta))],
    [0.25 * (1 + sp.cos(beta))**2, -0.5 * sp.sin(beta) * (1 + sp.cos(beta)), sp.sqrt(6)/4 * sp.sin(beta)**2, 
     -0.5 * sp.sin(beta) * (1 - sp.cos(beta)), 0.25 * (1 - sp.cos(beta))**2]
])

# Construct the full Wigner D-matrix for l=2
m_values = [-2, -1, 0, 1, 2]
D_2 = sp.zeros(5, 5)
for i, m1 in enumerate(m_values):
    for j, m2 in enumerate(m_values):
        D_2[i, j] = sp.exp(-sp.I * m1 * alpha) * d_2[i, j] * sp.exp(-sp.I * m2 * gamma)

# Define the transformation of T_{2m} under the Wigner D-matrix
T_transformed = sp.zeros(5, 1)
for i, m in enumerate(m_values):
    for j, m2 in enumerate(m_values):
        T_transformed[i] += D_2[i, j] * T_2m[m2]

# Display the transformed tensor coefficients
T_transformed_simplified = sp.simplify(T_transformed)
sp.pprint(T_transformed_simplified, use_unicode=True)


⎡⎛                               2  2⋅ⅈ⋅α   ⎛                        2⋅ⅈ⋅α     ↪
⎢⎝0.125⋅(δₓₓ - δ_yy)⋅(cos(β) - 1) ⋅ℯ      + ⎝0.306186217847897⋅√6⋅δ⋅ℯ     ⋅sin ↪
⎢                                                                              ↪
⎢ ⎛                                ⅈ⋅α   ⎛                        ⅈ⋅α          ↪
⎢ ⎝0.25⋅(δₓₓ - δ_yy)⋅(cos(β) - 1)⋅ℯ    + ⎝0.612372435695794⋅√6⋅δ⋅ℯ   ⋅cos(β) - ↪
⎢                                                                              ↪
⎢           ⎛                       2      ⎛                   ⎛       2       ↪
⎢           ⎝0.5⋅√6⋅(δₓₓ - δ_yy)⋅sin (β) + ⎝4.89897948556636⋅δ⋅⎝1.5⋅cos (β) -  ↪
⎢           ────────────────────────────────────────────────────────────────── ↪
⎢                                                                        4     ↪
⎢                                                                              ↪
⎢⎛                                  ⅈ⋅α   ⎛                                    ↪
⎢⎝- 0.25⋅(δₓₓ - δ_yy)⋅(cos(β

In [2]:
d_2

Matrix([
[       0.25*(cos(beta) + 1)**2,  0.5*(cos(beta) + 1)*sin(beta),        sqrt(6)*sin(beta)**2/4,  0.5*(1 - cos(beta))*sin(beta),        0.25*(1 - cos(beta))**2],
[-0.5*(cos(beta) + 1)*sin(beta),      (cos(beta) + 1)*cos(beta), sqrt(6)*sin(beta)*cos(beta)/2,      (1 - cos(beta))*cos(beta), -0.5*(1 - cos(beta))*sin(beta)],
[        sqrt(6)*sin(beta)**2/4, -sqrt(6)*sin(beta)*cos(beta)/2,        1.5*cos(beta)**2 - 0.5,  sqrt(6)*sin(beta)*cos(beta)/2,         sqrt(6)*sin(beta)**2/4],
[-0.5*(cos(beta) + 1)*sin(beta),     -(cos(beta) + 1)*cos(beta), sqrt(6)*sin(beta)*cos(beta)/2,     -(1 - cos(beta))*cos(beta),  0.5*(1 - cos(beta))*sin(beta)],
[       0.25*(cos(beta) + 1)**2, -0.5*(cos(beta) + 1)*sin(beta),        sqrt(6)*sin(beta)**2/4, -0.5*(1 - cos(beta))*sin(beta),        0.25*(1 - cos(beta))**2]])

In [3]:
D_2

Matrix([
[      0.25*(cos(beta) + 1)**2*exp(2*I*alpha)*exp(2*I*gamma),   0.5*(cos(beta) + 1)*exp(2*I*alpha)*exp(I*gamma)*sin(beta),       sqrt(6)*exp(2*I*alpha)*sin(beta)**2/4,   0.5*(1 - cos(beta))*exp(2*I*alpha)*exp(-I*gamma)*sin(beta),      0.25*(1 - cos(beta))**2*exp(2*I*alpha)*exp(-2*I*gamma)],
[ -0.5*(cos(beta) + 1)*exp(I*alpha)*exp(2*I*gamma)*sin(beta),         (cos(beta) + 1)*exp(I*alpha)*exp(I*gamma)*cos(beta),  sqrt(6)*exp(I*alpha)*sin(beta)*cos(beta)/2,         (1 - cos(beta))*exp(I*alpha)*exp(-I*gamma)*cos(beta), -0.5*(1 - cos(beta))*exp(I*alpha)*exp(-2*I*gamma)*sin(beta)],
[                      sqrt(6)*exp(2*I*gamma)*sin(beta)**2/4,                 -sqrt(6)*exp(I*gamma)*sin(beta)*cos(beta)/2,                      1.5*cos(beta)**2 - 0.5,                  sqrt(6)*exp(-I*gamma)*sin(beta)*cos(beta)/2,                      sqrt(6)*exp(-2*I*gamma)*sin(beta)**2/4],
[-0.5*(cos(beta) + 1)*exp(-I*alpha)*exp(2*I*gamma)*sin(beta),       -(cos(beta) + 1)*exp(-I*alpha)*exp(I*gamma)*cos

In [11]:
# calculate the correlation coefficients 
# multiplication of T_transformed_simplified 2 and 4 is the correlation coefficient
correlation_coefficient = T_transformed_simplified[1] * T_transformed_simplified[3]
correlation_coefficient_simplified = sp.simplify(correlation_coefficient)
sp.pprint(correlation_coefficient_simplified, use_unicode=True)


 ⎛                                ⅈ⋅α   ⎛                                      ↪
-⎝0.25⋅(δₓₓ - δ_yy)⋅(cos(β) - 1)⋅ℯ    - ⎝0.612372435695794⋅√6⋅δ⋅cos(β) - 0.25⋅ ↪

↪                            2⋅ⅈ⋅γ⎞  ⅈ⋅(α + 2⋅γ)⎞ ⎛                            ↪
↪ (δₓₓ - δ_yy)⋅(cos(β) + 1)⋅ℯ     ⎠⋅ℯ           ⎠⋅⎝0.25⋅(δₓₓ - δ_yy)⋅(cos(β) - ↪

↪      ⅈ⋅α   ⎛                        ⅈ⋅α                                      ↪
↪  1)⋅ℯ    + ⎝0.612372435695794⋅√6⋅δ⋅ℯ   ⋅cos(β) - 0.25⋅(δₓₓ - δ_yy)⋅(cos(β) + ↪

↪      ⅈ⋅(α + 2⋅γ)⎞  2⋅ⅈ⋅γ⎞  -2⋅ⅈ⋅(α + 2⋅γ)    2   
↪  1)⋅ℯ           ⎠⋅ℯ     ⎠⋅ℯ              ⋅sin (β)


In [9]:
# test if delta_xx and delta_yy are equal and gamma is zero
test_conditions = sp.simplify(sp.Eq(delta_xx, delta_yy) & sp.Eq(gamma, 0))
test_conditions_result = test_conditions.subs({delta_xx: 1, delta_yy: 1, gamma: 0})
#sp.pprint(test_conditions, use_unicode=True)
test_conditions_result_simplified = sp.simplify(test_conditions_result)
sp.pprint(test_conditions_result_simplified, use_unicode=True)

True


In [14]:
T_transformed_simplified[1]
# simplify T_transformed_simplified[1] with delata_xx = delta_yy and gamma = 0
simplified_T1 = sp.simplify(T_transformed_simplified[1].subs({delta_xx: delta_yy, gamma: 0}))
sp.pprint(simplified_T1, use_unicode=True)

                        ⅈ⋅α         
0.306186217847897⋅√6⋅δ⋅ℯ   ⋅sin(2⋅β)


In [15]:
T_transformed_simplified[3]
simplified_T3 = sp.simplify(T_transformed_simplified[3].subs({delta_xx: delta_yy, gamma: 0}))
sp.pprint(simplified_T3, use_unicode=True)

                        -ⅈ⋅α         
0.306186217847897⋅√6⋅δ⋅ℯ    ⋅sin(2⋅β)


In [16]:
T_transformed_simplified[3]
simplified_T2 = sp.simplify(T_transformed_simplified[2].subs({delta_xx: delta_yy, gamma: 0}))
sp.pprint(simplified_T2, use_unicode=True)

  ⎛                    2                       ⎞
δ⋅⎝1.83711730708738⋅cos (β) - 0.612372435695794⎠
